# 9.6 Assignment 9: Neural Networks


In [13]:
import pandas as pd
import numpy as np
from typing import List

import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
def print_unique(df):
    unique_values = {col: df[col].unique() for col in df.columns}
    for column, values in unique_values.items():
        print(f"Unique values in column '{column}': {values}")
        
def drop_cols(df: pd.DataFrame, cols_to_drop: List[str]):
    return df.drop(columns=cols_to_drop)

def convert_year_week_to_date(year_week):
    # Extract year and week from the input
    if isinstance(year_week, int):
        year_week_str = str(year_week)
    else:
        year_week_str = year_week
    year = int(year_week_str[:4])
    week = int(year_week_str[4:])
    # Get the first day of the week (Monday) for the given year and week
    first_date = datetime.strptime(f'{year}-W{week}-1', "%Y-W%W-%w").date()
    return first_date

In [4]:
import kagglehub
import os
def import_kaggle(urlpath:str, index=0):
    path = kagglehub.dataset_download(urlpath)
    files = os.listdir(path)
    print("Path to dataset files:", path)
    print(f"File names: {files}")
    # Checks how many files were downloaded and asks for index input if there's more than one
    if len(files) > 1:
        index = int(input("What's the index of the file you want?"))
    # If there's only 1 file, then it runs the dataframe on the index = 0
    else:
        pass
    file_name = files[index]
    df = pd.read_csv(rf"{path}\\{file_name}")
    print(f"\n{file_name} successfully loaded. Data shape: {df.shape}")
    return df

# use the function
df_steel = import_kaggle("uciml/faulty-steel-plates")
display(df_steel)

Path to dataset files: C:\Users\steve\.cache\kagglehub\datasets\uciml\faulty-steel-plates\versions\1
File names: ['faults.csv']

faults.csv successfully loaded. Data shape: (1941, 34)


,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,42,50,270900,270944,267,17,44,24220,76,108,1687,1,0,80,0.0498,0.2415,0.1818,0.0047,0.4706,1.0000,1.0,2.4265,0.9031,1.6435,0.8182,-0.2913,0.5822,1,0,0,0,0,0,0
1,645,651,2538079,2538108,108,10,30,11397,84,123,1687,1,0,80,0.7647,0.3793,0.2069,0.0036,0.6000,0.9667,1.0,2.0334,0.7782,1.4624,0.7931,-0.1756,0.2984,1,0,0,0,0,0,0
2,829,835,1553913,1553931,71,8,19,7972,99,125,1623,1,0,100,0.9710,0.3426,0.3333,0.0037,0.7500,0.9474,1.0,1.8513,0.7782,1.2553,0.6667,-0.1228,0.2150,1,0,0,0,0,0,0
3,853,860,369370,369415,176,13,45,18996,99,126,1353,0,1,290,0.7287,0.4413,0.1556,0.0052,0.5385,1.0000,1.0,2.2455,0.8451,1.6532,0.8444,-0.1568,0.5212,1,0,0,0,0,0,0
4,1289,1306,498078,498335,2409,60,260,246930,37,126,1353,0,1,185,0.0695,0.4486,0.0662,0.0126,0.2833,0.9885,1.0,3.3818,1.2305,2.4099,0.9338,-0.1992,1.0000,1,0,0,0,0,0,0
5,430,441,100250,100337,630,20,87,62357,64,127,1387,0,1,40,0.6200,0.3417,0.1264,0.0079,0.5500,1.0000,1.0,2.7993,1.0414,1.9395,0.8736,-0.2267,0.9874,1,0,0,0,0,0,0
6,413,446,138468,138883,9052,230,432,1481991,23,199,1687,0,1,150,0.4896,0.3390,0.0795,0.0196,0.1435,0.9607,1.0,3.9567,1.5185,2.6181,0.9205,0.2791,1.0000,1,0,0,0,0,0,0
7,190,200,210936,210956,132,11,20,20007,124,172,1687,0,1,150,0.2253,0.3400,0.5000,0.0059,0.9091,1.0000,1.0,2.1206,1.0000,1.3010,0.5000,0.1841,0.3359,1,0,0,0,0,0,0
8,330,343,429227,429253,264,15,26,29748,53,148,1687,0,1,150,0.3912,0.2189,0.5000,0.0077,0.8667,1.0000,1.0,2.4216,1.1139,1.4150,0.5000,-0.1197,0.5593,1,0,0,0,0,0,0
9,74,90,779144,779308,1506,46,167,180215,53,143,1687,0,1,150,0.0877,0.4261,0.0976,0.0095,0.3478,0.9820,1.0,3.1778,1.2041,2.2148,0.9024,-0.0651,1.0000,1,0,0,0,0,0,0


In [12]:
df_steel.dtypes

X_Minimum                  int64
X_Maximum                  int64
Y_Minimum                  int64
Y_Maximum                  int64
Pixels_Areas               int64
X_Perimeter                int64
Y_Perimeter                int64
Sum_of_Luminosity          int64
Minimum_of_Luminosity      int64
Maximum_of_Luminosity      int64
Length_of_Conveyer         int64
TypeOfSteel_A300           int64
TypeOfSteel_A400           int64
Steel_Plate_Thickness      int64
Edges_Index              float64
Empty_Index              float64
Square_Index             float64
Outside_X_Index          float64
Edges_X_Index            float64
Edges_Y_Index            float64
Outside_Global_Index     float64
LogOfAreas               float64
Log_X_Index              float64
Log_Y_Index              float64
Orientation_Index        float64
Luminosity_Index         float64
SigmoidOfAreas           float64
Pastry                     int64
Z_Scratch                  int64
K_Scratch                  int64
Stains    

## Dependent Variables (Faults in Steel Plates)
```
1	Pastry
2	Z_Scratch
3	K_Scratch
4	Stains
5	Dirtiness
6	Bumps
7	Other_Faults
```

## Independent Variables
```
1	X_Minimum	
2	X_Maximum	
3	Y_Minimum	
4	Y_Maximum	
5	Pixels_Areas	
6	X_Perimeter	
7	Y_Perimeter	
8	Sum_of_Luminosity	
9	Minimum_of_Luminosity	
10	Maximum_of_Luminosity	
11	Length_of_Conveyer	
12	TypeOfSteel_A300	
13	TypeOfSteel_A400	
14	Steel_Plate_Thickness
15	Edges_Index
16	Empty_Index
17	Square_Index
18	Outside_X_Index
19	Edges_X_Index
20	Edges_Y_Index
21	Outside_Global_Index
22	LogOfAreas
23	Log_X_Index
24	Log_Y_Index
25	Orientation_Index
26	Luminosity_Index
27	SigmoidOfAreas
```

In [5]:
df_steel = df_steel.rename(columns={'K_Scatch': 'K_Scratch'})
df_steel.columns

Index(['X_Minimum', 'X_Maximum', 'Y_Minimum', 'Y_Maximum', 'Pixels_Areas',
       'X_Perimeter', 'Y_Perimeter', 'Sum_of_Luminosity',
       'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer',
       'TypeOfSteel_A300', 'TypeOfSteel_A400', 'Steel_Plate_Thickness',
       'Edges_Index', 'Empty_Index', 'Square_Index', 'Outside_X_Index',
       'Edges_X_Index', 'Edges_Y_Index', 'Outside_Global_Index', 'LogOfAreas',
       'Log_X_Index', 'Log_Y_Index', 'Orientation_Index', 'Luminosity_Index',
       'SigmoidOfAreas', 'Pastry', 'Z_Scratch', 'K_Scratch', 'Stains',
       'Dirtiness', 'Bumps', 'Other_Faults'],
      dtype='object')

In [17]:
independent_vars = [
    "X_Minimum", "X_Maximum", "Y_Minimum", "Y_Maximum", "Pixels_Areas",
    "X_Perimeter", "Y_Perimeter", "Sum_of_Luminosity", "Minimum_of_Luminosity",
    "Maximum_of_Luminosity", "Length_of_Conveyer", "TypeOfSteel_A300",
    "TypeOfSteel_A400", "Steel_Plate_Thickness", "Edges_Index", "Empty_Index",
    "Square_Index", "Outside_X_Index", "Edges_X_Index", "Edges_Y_Index",
    "Outside_Global_Index", "LogOfAreas", "Log_X_Index", "Log_Y_Index",
    "Orientation_Index", "Luminosity_Index", "SigmoidOfAreas"
]

# Predictors
fault_type_classes = [
    "Pastry", "Z_Scratch", "K_Scratch", "Stains",
    "Dirtiness", "Bumps", "Other_Faults"
]

X = df_steel[independent_vars]
y = df_steel[fault_type_classes]

print(f"Features (X) shape: {X.shape}")
print(f"Target (y) shape: {y.shape}")
print(f"Number of unique fault types in y: \n{y.nunique()}")
print(f"\nFault classes to predict: {fault_type_classes}")
# Neural networks generally perform better with scaled features.
print("\nScaling features using StandardScaler...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# X_scaled is a NumPy array, which is suitable for MLPClassifier.

print("Splitting data into training and testing sets...")
# We use X_scaled (NumPy array) and y (Pandas Series)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.25, random_state=42, stratify=y
)
# stratify=y ensures that the class distribution is similar in train and test sets.

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

# --- Initialize and Train the MLP Model ---
print("\nInitializing and training the MLPClassifier model...")
# Define the MLP model
# You can tune these hyperparameters
mlp_model = MLPClassifier(
    hidden_layer_sizes=(128, 64, 32), # Example: 3 hidden layers
    activation='relu',                # Activation function for hidden layers
    solver='adam',                    # Optimizer
    alpha=0.0001,                     # L2 penalty (regularization)
    batch_size='auto',                # Size of minibatches for stochastic optimizers
    learning_rate_init=0.001,
    max_iter=500, # Added for convergence
    random_state=42
)
mlp_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = mlp_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")

Features (X) shape: (1941, 27)
Target (y) shape: (1941, 7)
Number of unique fault types in y: 
Pastry          2
Z_Scratch       2
K_Scratch       2
Stains          2
Dirtiness       2
Bumps           2
Other_Faults    2
dtype: int64

Fault classes to predict: ['Pastry', 'Z_Scratch', 'K_Scratch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']

Scaling features using StandardScaler...
Splitting data into training and testing sets...
X_train shape: (1455, 27), y_train shape: (1455, 7)
X_test shape: (486, 27), y_test shape: (486, 7)

Initializing and training the MLPClassifier model...
Model Accuracy: 0.7099


Defaults for MLPClassifier:
```
class sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(100,), activation='relu', *, solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000)
```

Going to try these different hidden layer sizes:
 - Fewer layers, more neurons: (256, 128)
 - More layers, fewer neurons: (128, 64, 32, 16)
 - Different sizes: (100, 50, 25)

In [9]:
for hls in [(256, 128), (128, 64, 32, 16), (100, 50, 25)]:
    print(f"Using hidden_layer_sizes={hls}")
    mlp_model = MLPClassifier(
        hidden_layer_sizes=hls, # Example: 3 hidden layers
        activation='relu',                # Activation function for hidden layers
        solver='adam',                    # Optimizer
        alpha=0.0001,                     # L2 penalty (regularization)
        batch_size='auto',                # Size of minibatches for stochastic optimizers
        learning_rate_init=0.001,
        max_iter=500, # Added for convergence
        random_state=42
    )
    mlp_model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = mlp_model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model Accuracy: {accuracy:.4f}")

Using hidden_layer_sizes=(256, 128)
Model Accuracy: 0.7202
Using hidden_layer_sizes=(128, 64, 32, 16)
Model Accuracy: 0.7119
Using hidden_layer_sizes=(100, 50, 25)
Model Accuracy: 0.7160


In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np 

param_grid = {
    'estimator__hidden_layer_sizes': [(128, 64, 32), (256, 128), (100, 50)],
    'estimator__activation': ['relu', 'tanh'],
    'estimator__solver': ['adam', 'sgd'],
    'estimator__alpha': [0.0001, 0.0005, 0.001],
    'estimator__learning_rate_init': [0.001, 0.0005],
}
base_mlp_grid = MLPClassifier(random_state=42, max_iter=500) # Set a base max_iter for grid search
multi_output_mlp_grid = MultiOutputClassifier(estimator=base_mlp_grid, n_jobs=-1)
grid_search = GridSearchCV(multi_output_mlp_grid, param_grid, cv=3, verbose=2, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)

print(f"Best parameters found: {grid_search.best_params_}")
print(f"Best cross-validation exact match accuracy: {grid_search.best_score_:.4f}") # This might be averaged per label by MultiOutputClassifier

best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
final_accuracy = np.all(y_pred_best == y_test, axis=1).mean()
print(f"Final Model Exact Match Accuracy on Test Set: {final_accuracy:.4f}")

Fitting 3 folds for each of 72 candidates, totalling 216 fits
Best parameters found: {'estimator__activation': 'tanh', 'estimator__alpha': 0.0001, 'estimator__hidden_layer_sizes': (256, 128), 'estimator__learning_rate_init': 0.001, 'estimator__solver': 'adam'}
Best cross-validation exact match accuracy: 0.6701
Final Model Exact Match Accuracy on Test Set: 0.7202


# Analysis
An MLP Classifier Model was built to predict the type of faults in steel plates<p>
All features were used because they were all numeric.<p>
First the Standard Scaler was used on the features to normalize them. The data was split 75/25 train/test. Then the data was put through a Multi-layer Perceptron (MLP) classifier with fairly standard parameters resulting in an accuracy of about 70%. Adjusting the Random State to various values resulted in accuracies between 69% and 72% which confirms that the baseline model is reasonably stable and not overly sensitive to the initial random weights.<p>
This first model had a fairly good result, but several techniques were used to see if we could improve on that.<p>
Next we tried manipulating the hidden layers:<p>
 - Fewer layers, more neurons: (256, 128)
 - More layers, fewer neurons: (128, 64, 32, 16)
 - Different sizes: (100, 50, 25)
This resulted in accuracies of 71%-72%, basically the same as in previous parameters. I've likely found a configuration that's close to optimal for this aspect of the model, given the current features.<p>
Finally I tried a Grid Search with different permutations of parameters. This resulted in accuracies of 67%-72% with the highest matching the very first model fit with `hidden_layer_sizes = (256, 128)`.<p>
Within the parameter space explored, the model is likely operating near its performance ceiling. The search didn't uncover a "magic bullet" combination of parameters that dramatically boosts accuracy.<p>
The current set of 27 numerical features, even after standardization, may not contain enough inherent information or distinct patterns to allow the MLP to achieve significantly higher accuracy for this multi-label classification task.<p>
An accuracy of 70−72% often refers to "exact match" accuracy (where all 7 predictions must be correct for a sample to count as accurate). This is a very strict metric, and achieving over 70% is a decent result under such conditions.<p>